In [1]:
import pandas as pd
import numpy as np
import astropy.units as u
from astropy.constants import hbar, c, eps0, e, a0

from LightMat import Atom

In [2]:
hfs_state = {'n': 4, 'L': 's', 'J': 1/2, 'F': 9/2, 'mF': 4}

atom = Atom('K', hfs_state=hfs_state, I=4)

In [12]:
(-1)**(0+ 9/2 + 1 + 9/2)

1.0

In [3]:
atom.fs_transition_data

,transition,wavelength,reduced_dipole_element,linewidth
0,"{'n': 4, 'L': 'p', 'J': 0.5}",770.1084,4.106004,5.951683
1,"{'n': 4, 'L': 'p', 'J': 1.5}",766.7009,5.807005,6.031887
2,"{'n': 5, 'L': 'p', 'J': 0.5}",404.8356,0.275830,1.167693
3,"{'n': 5, 'L': 'p', 'J': 1.5}",404.5285,0.405932,1.195178
4,"{'n': 6, 'L': 'p', 'J': 0.5}",344.8363,0.083927,0.466356
5,"{'n': 6, 'L': 'p', 'J': 1.5}",344.7359,0.127649,0.478356
6,"{'n': 7, 'L': 'p', 'J': 0.5}",321.8549,0.039180,0.243587
7,"{'n': 7, 'L': 'p', 'J': 1.5}",321.8083,0.061411,0.249875
8,"{'n': 8, 'L': 'p', 'J': 0.5}",310.2946,0.022602,0.145519
9,"{'n': 8, 'L': 'p', 'J': 1.5}",310.2689,0.036377,0.149232


In [3]:
# Read atomic spectra data from csv file
csv_file_path = './LightMat/matter/atomic_data/K1TransitionRates.csv'
df = pd.read_csv(csv_file_path, usecols=[0, 1, 5, 8,])

# Rename columns
df = df.rename(columns={
    'Initial': 'initial', 
    'Final': 'final', 
    'Wavelength (nm)': 'wavelength', 
    'Transition rate (s-1)': 'transition_rate',
    })

# Convert initial and final states from string to dict {'n': int, 'J': float}
def fraction_to_float(fraction_str):
    numerator, denominator = fraction_str.split('/')
    return float(numerator) / float(denominator)

pattern = r'(\d+)([a-zA-Z]+)(\d+/\d+)'
df['initial'] = df['initial'].str.extract(pattern).apply(lambda x: {
    'n': int(x[0]), 
    'L': x[1],
    'J': fraction_to_float(x[2]), 
    }, axis=1)
df['final'] = df['final'].str.extract(pattern).apply(lambda x: {
    'n': int(x[0]), 
    'L': x[1],
    'J': fraction_to_float(x[2]), 
    }, axis=1)


# Create a new DataFrame with 'transitions' column
filtered_df = df[df['initial'].apply(lambda x: x == fs_state) | df['final'].apply(lambda x: x == fs_state)]
transitions = filtered_df.apply(lambda row: row['initial'] if row['final'] == fs_state else row['final'], axis=1)
fs_transitions_df = filtered_df.copy()
fs_transitions_df['transition'] = transitions
fs_transitions_df = fs_transitions_df.drop(columns=['initial', 'final'])
fs_transitions_df = fs_transitions_df[['transition', 'wavelength', 'transition_rate']]
fs_transitions_df.reset_index(drop=True, inplace=True)


# Calculate the reduced dipole element 
J_primes = np.asarray(fs_transitions_df['transition'].apply(lambda x: x['J']))
wavelengths = np.asarray(fs_transitions_df['wavelength'].values) * u.nm
omegas = 2 * np.pi * c / wavelengths
transition_rates = np.asarray(fs_transitions_df['transition_rate'].values) * 1/u.s


reduced_dipole_elements = np.sqrt(
    (transition_rates * 3 * np.pi * eps0 * hbar * c**3 * (2 * J_primes + 1)) / (omegas**3)
).to(u.C * u.m)
reduced_dipole_elements_au = reduced_dipole_elements / (e.si * a0)

fs_transitions_df['reduced_dipole_element'] = reduced_dipole_elements_au.value


# Calculate the transition linewidths
linewidth_fs_state = (df[df['initial'].apply(lambda x: x == fs_state)]['transition_rate'].sum() / (2*np.pi) * u.Hz).to(u.MHz).value

linewidths = []
for state in fs_transitions_df['transition']:
    linewidth_state = (df[df['initial'].apply(lambda x: x == state)]['transition_rate'].sum() / (2*np.pi) * u.Hz).to(u.MHz).value
    linewidths.append(linewidth_state + linewidth_fs_state)

fs_transitions_df['linewidth'] = linewidths
fs_transitions_df = fs_transitions_df.drop(columns=['transition_rate'])

NameError: name 'fs_state' is not defined

In [ ]:
fs_transitions_df

,transition,wavelength,reduced_dipole_element,linewidth
0,"{'n': 4, 'L': 'p', 'J': 0.5}",770.1084,4.106004,5.951683
1,"{'n': 4, 'L': 'p', 'J': 1.5}",766.7009,5.807005,6.031887
2,"{'n': 5, 'L': 'p', 'J': 0.5}",404.8356,0.275830,1.167693
3,"{'n': 5, 'L': 'p', 'J': 1.5}",404.5285,0.405932,1.195178
4,"{'n': 6, 'L': 'p', 'J': 0.5}",344.8363,0.083927,0.466356
5,"{'n': 6, 'L': 'p', 'J': 1.5}",344.7359,0.127649,0.478356
6,"{'n': 7, 'L': 'p', 'J': 0.5}",321.8549,0.039180,0.243587
7,"{'n': 7, 'L': 'p', 'J': 1.5}",321.8083,0.061411,0.249875
8,"{'n': 8, 'L': 'p', 'J': 0.5}",310.2946,0.022602,0.145519
9,"{'n': 8, 'L': 'p', 'J': 1.5}",310.2689,0.036377,0.149232


In [ ]:
def hfs(a, b, I, J):
    E = {}
    for F in np.linspace(abs(I-J),I+J, int(2*J+1)):
        G = F*(F+1) - I*(I+1) - J*(J+1)
        if b.value == 0:
            E[F] = (0.5*a*G / h).to(u.MHz)
        else:
            E[F] = ((0.5*a*G + b*(1.5*G*(G+1)-2*I*(I+1)*J*(J+1))/(2*I*(2*I-1)*2*J*(2*J-1))) / h).to(u.MHz)

    return E